## Ninja Gaiden
Forex scalping backtest program.

Tkinter interface, asks the user what forex pair they want data from yfinance on. 
After data is loaded, user selects which backtest stratey they want to deploy. 
Backtest is run and output is displayed on html.  All coded within one python script. 

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import webbrowser
import os

class ForexBacktester:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Forex Backtesting System")
        self.root.geometry("800x600")
        self.root.configure(bg='#1e1e1e')
        
        # Style configuration
        style = ttk.Style()
        style.theme_use('default')
        style.configure("TLabel", background='#1e1e1e', foreground='white')
        style.configure("TButton", background='#333333', foreground='white')
        style.configure("TCombobox", background='#333333', foreground='white')
        
        self.create_widgets()
        
    def create_widgets(self):
        # Forex Pair Selection
        pair_frame = ttk.Frame(self.root)
        pair_frame.pack(pady=20)
        
        ttk.Label(pair_frame, text="Select Forex Pair:").pack(side=tk.LEFT, padx=5)
        self.pair_var = tk.StringVar()
        pairs = ['EUR/USD', 'GBP/USD', 'USD/JPY', 'USD/CHF', 'AUD/USD', 'USD/CAD']
        self.pair_combo = ttk.Combobox(pair_frame, textvariable=self.pair_var, values=pairs)
        self.pair_combo.set('EUR/USD')
        self.pair_combo.pack(side=tk.LEFT, padx=5)
        
        # Date Range
        date_frame = ttk.Frame(self.root)
        date_frame.pack(pady=10)
        
        ttk.Label(date_frame, text="Days of Historical Data:").pack(side=tk.LEFT, padx=5)
        self.days_var = tk.StringVar(value="30")
        days_entry = ttk.Entry(date_frame, textvariable=self.days_var, width=10)
        days_entry.pack(side=tk.LEFT, padx=5)
        
        # Load Data Button
        load_btn = ttk.Button(self.root, text="Load Data", command=self.load_data)
        load_btn.pack(pady=10)
        
        # Strategy Selection
        strategy_frame = ttk.Frame(self.root)
        strategy_frame.pack(pady=20)
        
        ttk.Label(strategy_frame, text="Select Strategy:").pack(side=tk.LEFT, padx=5)
        self.strategy_var = tk.StringVar()
        strategies = ['Moving Average Crossover', 'RSI Strategy', 'Bollinger Bands']
        self.strategy_combo = ttk.Combobox(strategy_frame, textvariable=self.strategy_var, values=strategies)
        self.strategy_combo.set('Moving Average Crossover')
        self.strategy_combo.pack(side=tk.LEFT, padx=5)
        
        # Run Backtest Button
        self.backtest_btn = ttk.Button(self.root, text="Run Backtest", command=self.run_backtest)
        self.backtest_btn.pack(pady=10)
        self.backtest_btn.config(state='disabled')
        
    def load_data(self):
        try:
            pair = self.pair_var.get().replace('/', '')
            days = int(self.days_var.get())
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)
            
            self.data = yf.download(f"{pair}=X", start=start_date, end=end_date, interval='1h')
            
            if len(self.data) > 0:
                messagebox.showinfo("Success", f"Loaded {len(self.data)} data points for {self.pair_var.get()}")
                self.backtest_btn.config(state='normal')
            else:
                messagebox.showerror("Error", "No data found for selected pair")
                
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load data: {str(e)}")
    
    def moving_average_crossover(self, data):
        df = data.copy()
        df['SMA20'] = df['Close'].rolling(window=20).mean()
        df['SMA50'] = df['Close'].rolling(window=50).mean()
        
        df['Signal'] = 0
        df.loc[df['SMA20'] > df['SMA50'], 'Signal'] = 1
        df.loc[df['SMA20'] < df['SMA50'], 'Signal'] = -1
        
        df['Returns'] = df['Close'].pct_change() * df['Signal'].shift(1)
        return df
    
    def rsi_strategy(self, data):
        df = data.copy()
        # Calculate RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        df['Signal'] = 0
        df.loc[df['RSI'] < 30, 'Signal'] = 1
        df.loc[df['RSI'] > 70, 'Signal'] = -1
        
        df['Returns'] = df['Close'].pct_change() * df['Signal'].shift(1)
        return df
    
    def bollinger_bands(self, data):
        df = data.copy()
        df['SMA20'] = df['Close'].rolling(window=20).mean()
        df['STD'] = df['Close'].rolling(window=20).std()
        df['UpperBand'] = df['SMA20'] + (df['STD'] * 2)
        df['LowerBand'] = df['SMA20'] - (df['STD'] * 2)
        
        df['Signal'] = 0
        df.loc[df['Close'] < df['LowerBand'], 'Signal'] = 1
        df.loc[df['Close'] > df['UpperBand'], 'Signal'] = -1
        
        df['Returns'] = df['Close'].pct_change() * df['Signal'].shift(1)
        return df
    
    def run_backtest(self):
        strategy = self.strategy_var.get()
        
        if strategy == 'Moving Average Crossover':
            results = self.moving_average_crossover(self.data)
        elif strategy == 'RSI Strategy':
            results = self.rsi_strategy(self.data)
        else:
            results = self.bollinger_bands(self.data)
        
        # Calculate performance metrics
        total_returns = results['Returns'].sum() * 100
        sharpe_ratio = np.sqrt(252) * (results['Returns'].mean() / results['Returns'].std())
        max_drawdown = (results['Close'] / results['Close'].cummax() - 1).min() * 100
        
        # Generate HTML report
        html_content = f"""
        <html>
        <head>
            <title>Backtest Results</title>
            <style>
                body {{
                    background-color: #1e1e1e;
                    color: white;
                    font-family: Arial, sans-serif;
                    margin: 20px;
                }}
                .container {{
                    max-width: 800px;
                    margin: 0 auto;
                }}
                .metric {{
                    background-color: #333333;
                    padding: 15px;
                    margin: 10px 0;
                    border-radius: 5px;
                }}
                h1, h2 {{
                    color: #00ff00;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Backtest Results</h1>
                <h2>Strategy: {strategy}</h2>
                <h2>Pair: {self.pair_var.get()}</h2>
                
                <div class="metric">
                    <h3>Total Returns: {total_returns:.2f}%</h3>
                </div>
                
                <div class="metric">
                    <h3>Sharpe Ratio: {sharpe_ratio:.2f}</h3>
                </div>
                
                <div class="metric">
                    <h3>Maximum Drawdown: {max_drawdown:.2f}%</h3>
                </div>
                
                <div class="metric">
                    <h3>Number of Trades: {len(results[results['Signal'] != 0])}</h3>
                </div>
            </div>
        </body>
        </html>
        """
        
        # Save and open HTML report
        with open('backtest_results.html', 'w') as f:
            f.write(html_content)
        
        webbrowser.open('file://' + os.path.realpath('backtest_results.html'))
    
    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    app = ForexBacktester()
    app.run()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
